In [ ]:
model_name = "bloomz-7b1mt"
from transformers import AutoTokenizer,AutoModelForCausalLM,GenerationConfig,AutoModel
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda")
MODEL = 'bloomz'

In [ ]:
from baukit import Trace, TraceDict

def get_out_bloomz(model, prompt, device,index): 

    model.eval()
    MLP_act = [f"transformer.h.{i}.mlp.gelu_impl" for i in range(30)]
    
    with torch.no_grad():
        with TraceDict(model, MLP_act) as ret:
            output = model(prompt, output_hidden_states = True,output_attentions=True)
        MLP_act_value = [ret[act_value].output for act_value in MLP_act]
        return MLP_act_value
    
    
def act_bloom(input_ids):
    mlp_act = get_out_bloomz(model,input_ids,model.device,-1)
    mlp_act = np.array(mlp_act)
    return mlp_act

if 'bloom' in MODEL:
    LAYERS = model.config.n_layer
    Neuron_num = 16384


In [ ]:
langs = ['en','de','es','fr','ru','th','tr','vi','zh','pt']
colors = ['cornflowerblue','forestgreen','orange','red','mediumturquoise','midnightblue','brown','moccasin','darkviolet','gold','deeppink','gray','teal','slateblue']

## XNLI data

In [ ]:
def load_data_XNLI():
    with open("./mnli/xnli.test.tsv") as f:
        lines = f.readlines()
    copora = [[],[],[],[],[]]
    for line in lines[1:]:
        line = line.split('\t')
        lang,label,sent1,sent2 = line[0],line[1],line[6],line[7]
        if lang in langs:
            ind = langs.index(lang)
            res = (label,sent1,sent2)
            copora[ind].append(res)
        
    length = len(copora[0])
    question_all, answer_all = [],[]
    for i in range(length):
        question_all.append([])
        answer_all.append([])
    for ind in range(length):
        for l in range(len(copora)):
            tup = copora[l]
            label,premise,hypothesis = tup[ind][0],tup[ind][1],tup[ind][2]
            prompt = f'Take the following as truth: {premise}\nThen the following statement: "{hypothesis}" is "true", "false", or "inconclusive"?'
            question_all[ind].append(prompt)
            answer_all[ind].append(label)
    question_all, answer_all = question_all[:1000], answer_all[:1000]    
    return question_all,answer_all

## behaviors of four types of neurons

In [ ]:
from numpy import *
import torch
import numpy as np

from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import matplotlib.pyplot as plt
from matplotlib.pyplot import MultipleLocator
import matplotlib.ticker as ticker

question_all,answer_all = load_data_XNLI()

test_counts = len(question_all)
langs_counts = len(question_all[0])

all_activated_counts,all_shared_counts,non_counts,specific_counts,all_partial_shared_counts = [],[],[],[] ,[]
for ind in range(test_counts):
    sents = question_all[ind]
    mlp_acts = []
    for lang in range(len(sents)):
        question = sents[lang]
        encodings = tokenizer(question, return_tensors='pt').to('cuda')
        input_ids = encodings['input_ids'].to('cuda')

        if 'bloom' in MODEL:
            mlp_act = act_bloom(input_ids)

        mlp_acts.append(mlp_act)

    mlp_all_act = []
    for i in range(langs_counts):
        mlp_act = (mlp_acts[i]>0).astype(int)
        mlp_all_act.append(mlp_act)
    mlp_all = np.sum(mlp_all_act,axis=0)    

    all_activated,all_activated_count = [],[]
    for i in range(langs_counts):   
        activated,activated_count = [],[]
        for ly in range(LAYERS):
            act = torch.nonzero(torch.tensor(mlp_all_act[i][ly]==1)).squeeze(1).detach().cpu().numpy()
            activated.append(act) 
            activated_count.append(len(act))
        all_activated.append(activated)
        all_activated_count.append(activated_count)


    all_shared,all_shared_count = [],[]
    for ly in range(LAYERS):
        mlp_all_act_inter = torch.nonzero(torch.tensor(mlp_all[ly]==langs_counts)).squeeze(1).detach().cpu().numpy()
        all_shared.append(mlp_all_act_inter)
        all_shared_count.append(len(mlp_all_act_inter))

    non,non_count = [],[]
    mlp_all = np.sum(mlp_all_act,axis=0)
    for ly in range(LAYERS):
        mlp_all_act_inter = torch.nonzero(torch.tensor(mlp_all[ly]==0)).squeeze(1).detach().cpu().numpy()
        non.append(mlp_all_act_inter)  
        non_count.append(len(mlp_all_act_inter))


    specific,specific_count = [],[]
    for i in range(langs_counts):       
        indices_arr1 = np.where(mlp_all_act[i] == 1)
        indices_arr2 = np.where(mlp_all == 1)
        set_indices_arr1 = set(zip(indices_arr1[0], indices_arr1[1]))
        set_indices_arr2 = set(zip(indices_arr2[0], indices_arr2[1]))
        intersection = set_indices_arr1.intersection(set_indices_arr2)
        if len(intersection) > 0:
            rows_intersection, cols_intersection = zip(*intersection)
        else:
            rows_intersection, cols_intersection = [],[]
        row = [[] for _ in range(LAYERS)]
        for k in range(len(rows_intersection)):
            r,c = rows_intersection[k],cols_intersection[k]
            row[r].append(c)
        specific.append(row)


    all_specific,specific_count = [],[]
    for ly in range(LAYERS):
        temp = []
        for i in range(langs_counts):
            temp += specific[i][ly]
        all_specific.append(temp)
        specific_count.append(len(temp))



    # some shared neuron
    partial_shared = []
    for lg in range(langs_counts):
        lg_some = []
        for ly in range(LAYERS):
            other = all_shared[ly].tolist() + specific[lg][ly]
            some = list((set(all_activated[lg][ly])-set(other)))
            lg_some.append(some)
        partial_shared.append(lg_some)

    all_partial_shared,all_partial_shared_count = [],[]
    for ly in range(LAYERS):
        temp = []
        for i in range(langs_counts):
            temp += partial_shared[i][ly]
        all_partial_shared.append(list(set(temp)))
        all_partial_shared_count.append(len(list(set(temp))))

    all_activated_counts.append(all_activated_count)
    all_shared_counts.append(all_shared_count)
    non_counts.append(non_count)
    specific_counts.append(specific_count)
    all_partial_shared_counts.append(all_partial_shared_count)


print(len(all_activated_counts))

all_activated_counts,all_shared_counts,non_counts,specific_counts,all_partial_shared_counts = mean(all_activated_counts, axis=0),mean(all_shared_counts, axis=0),mean(non_counts, axis=0),mean(specific_counts, axis=0),mean(all_partial_shared_counts, axis=0)
print(len(all_activated_counts),len(all_activated_counts))    


legend_font = {
'style': 'normal',
'size': 6.5,
'weight': "bold", 
}

# Create the figure
fig, ax = plt.subplots(1,2,figsize=(4,2))


xLabel=[i+1 for i in range(LAYERS//2)]
x = [(i+1)*2 for i in range(LAYERS//2)]

ax[0].plot(x,np.array(non_counts[::2])/(Neuron_num/100),colors[0],marker='s',markersize='2.5',label='non-activated')   


ax[0].plot(x,np.array(specific_counts[::2])/(Neuron_num/100),colors[3],marker='o', markersize='2.5', label='specific')   
ax[0].plot(x,np.array(all_shared_counts[::2])/(Neuron_num/100),colors[1],marker='d',markersize='2.5', label='all-shared')
ax[0].plot(x,np.array(all_partial_shared_counts[::2])/(Neuron_num/100),colors[2],marker='<',markersize='2.5', label='partial-shared')

ax[0].set_title('by neuron type',fontsize=7,fontweight='bold')
ax[0].tick_params(labelsize=8)  
ax[0].xaxis.set_major_locator(ticker.MultipleLocator(5))
ax[0].legend(bbox_to_anchor=(0, -0.15), loc=2, ncol=1,prop=legend_font)


for k in range(len(all_activated_counts)):
    ax[1].plot(x,np.array(all_activated_counts[k][::2])/(Neuron_num/100),colors[k+4],marker='o', linestyle='-',markersize='2', label=langs[k])    

ax[1].set_title('by language',fontsize=7,fontweight='bold')
ax[1].tick_params(labelsize=8)
ax[1].xaxis.set_major_locator(ticker.MultipleLocator(5))
ax[1].legend(bbox_to_anchor=(-0.1, -0.15), loc=2, ncol=3,prop=legend_font)


fig.text(0.53, -0.01, 'Layers', ha='center',fontsize=8,fontweight='bold')
fig.text(0.03, 0.5, 'Percentage', va='center', rotation='vertical',fontsize=8,fontweight='bold')
fig.subplots_adjust(wspace=0.25,hspace=0.35) 

plt.show()

# overlap across testset

In [ ]:
from numpy import *
import torch
import numpy as np

from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import matplotlib.pyplot as plt
from matplotlib.pyplot import MultipleLocator
import matplotlib.ticker as ticker


question_all,answer_all = load_data_XNLI()

test_counts = len(question_all)
langs_counts = len(question_all[0])

all_activated_counts,all_shared_counts,non_counts,specific_counts,all_partial_shared_counts = [],[],[],[] ,[]
all_shared_testset,non_testset,specific_testset,partial_shared_testset = [],[],[],[]
for ind in range(test_counts):
    sents = question_all[ind]
    mlp_acts = []
    for lang in range(len(sents)):
        question = sents[lang]
        encodings = tokenizer(question, return_tensors='pt')
        input_ids = encodings['input_ids'].to('cuda')

        if 'bloom' in model_name:
            mlp_act = act_bloom(input_ids)
        mlp_acts.append(mlp_act)

    mlp_all_act = []
    for i in range(langs_counts):
        mlp_act = (mlp_acts[i]>0).astype(int)
        mlp_all_act.append(mlp_act)
    mlp_all = np.sum(mlp_all_act,axis=0)    

    # monolingual activated
    all_activated,all_activated_count = [],[]
    for i in range(langs_counts):   
        activated,activated_count = [],[]
        for ly in range(LAYERS):
            act = torch.nonzero(torch.tensor(mlp_all_act[i][ly]==1)).squeeze(1).detach().cpu().numpy()
            activated.append(act) 
            activated_count.append(len(act))
        all_activated.append(activated)
        all_activated_count.append(activated_count)

    all_shared,all_shared_count,all_shared_index = [],[],[]
    for ly in range(LAYERS):
        mlp_all_act_inter = torch.nonzero(torch.tensor(mlp_all[ly]==langs_counts)).squeeze(1).detach().cpu().numpy()
        all_shared.append(mlp_all_act_inter)
        all_shared_count.append(len(mlp_all_act_inter))

    shared_index = (mlp_all==langs_counts).astype(int)
    non_index = (mlp_all==0).astype(int)
    all_shared_testset.append(shared_index)
    non_testset.append(non_index)

    non,non_count = [],[]
    mlp_all = np.sum(mlp_all_act,axis=0)
    for ly in range(LAYERS):
        mlp_all_act_inter = torch.nonzero(torch.tensor(mlp_all[ly]==0)).squeeze(1).detach().cpu().numpy()
        non.append(mlp_all_act_inter)  
        non_count.append(len(mlp_all_act_inter))

    specific,specific_count = [],[]
    for i in range(langs_counts):       
        indices_arr1 = np.where(mlp_all_act[i] == 1)
        indices_arr2 = np.where(mlp_all == 1)
        set_indices_arr1 = set(zip(indices_arr1[0], indices_arr1[1]))
        set_indices_arr2 = set(zip(indices_arr2[0], indices_arr2[1]))
        intersection = set_indices_arr1.intersection(set_indices_arr2)
        if len(intersection) > 0:
            rows_intersection, cols_intersection = zip(*intersection)
        else:
            rows_intersection, cols_intersection = [],[]
        row = [[] for _ in range(LAYERS)]
        for k in range(len(rows_intersection)):
            r,c = rows_intersection[k],cols_intersection[k]
            row[r].append(c)
        specific.append(row)


    all_specific,specific_count = [],[]
    specific_index = [[0 for _ in range(Neuron_num)] for _ in range(LAYERS)]
    for ly in range(LAYERS):
        temp = []
        for i in range(langs_counts):
            temp += specific[i][ly]
            for index in specific[i][ly]:
                specific_index[ly][index] = 1
        all_specific.append(temp)
        specific_count.append(len(temp))
    specific_testset.append(specific_index)


    partial_shared = []
    some_index = [[0 for _ in range(Neuron_num)] for _ in range(LAYERS)]
    for lg in range(langs_counts):
        lg_some = []
        for ly in range(LAYERS):
            other = all_shared[ly].tolist() + specific[lg][ly]
            some = list((set(all_activated[lg][ly])-set(other)))
            lg_some.append(some)
        partial_shared.append(lg_some)

    all_partial_shared,all_partial_shared_count = [],[]
    for ly in range(LAYERS):
        temp = []
        for i in range(langs_counts):
            temp += partial_shared[i][ly]
            for index in partial_shared[i][ly]:
                some_index[ly][index] = 1
        all_partial_shared.append(list(set(temp)))
        all_partial_shared_count.append(len(list(set(temp)))) 
    partial_shared_testset.append(some_index)


    all_activated_counts.append(all_activated_count)
    all_shared_counts.append(all_shared_count)
    non_counts.append(non_count)
    specific_counts.append(specific_count)
    all_partial_shared_counts.append(all_partial_shared_count)


avg_all_activated_counts,avg_all_shared_counts,avg_non_counts,avg_specific_counts,avg_all_partial_shared_counts = mean(all_activated_counts, axis=0),mean(all_shared_counts, axis=0),mean(non_counts, axis=0),mean(specific_counts, axis=0),mean(all_partial_shared_counts, axis=0)
sum_all_shared_counts,sum_non_counts,sum_specific_counts,sum_all_partial_shared_counts = np.sum(all_shared_testset, axis=0),np.sum(non_testset, axis=0),np.sum(specific_testset, axis=0),np.sum(partial_shared_testset, axis=0)

num = test_counts
counts_testset_all_shared,counts_testset_non,counts_testset_specific,counts_testset_partial_shared = [],[],[],[]
for j in range(LAYERS):
    counts_testset_all_shared.append(sum_all_shared_counts[j].tolist().count(num)) 
    counts_testset_non.append(sum_non_counts[j].tolist().count(num)) 
    counts_testset_specific.append(sum_specific_counts[j].tolist().count(num)) 
    counts_testset_partial_shared.append(sum_all_partial_shared_counts[j].tolist().count(num)) 



legend_font = {
'style': 'normal',
'size': 6.5,  # 字号
'weight': "bold",  # 是否加粗，不加粗
}
legend_elements = [
                       Line2D([0], [0], marker='o', color='w', label='overlap',markerfacecolor='#37A2FF'),
                       Line2D([0], [0], marker='o', color='w', label='average',markerfacecolor='#A17DB4'),
                      ]


# Create the figure
fig, ax = plt.subplots(2,2,figsize=(4,4))


xLabel=[i+1 for i in range(LAYERS//2)]
x = [(i+1)*2 for i in range(LAYERS//2)]#点的横坐标


ax[0,0].stackplot(x,np.array(counts_testset_all_shared[::2])/(Neuron_num/100),np.array(avg_all_shared_counts[::2]-counts_testset_all_shared[::2])/(Neuron_num/100),colors=['#37A2FF','#A17DB4'], alpha=0.5)

ax[0,0].set_title('All-shared',fontsize=7,fontweight='bold')
ax[0,0].tick_params(labelsize=8)  
ax[0,0].xaxis.set_major_locator(ticker.MultipleLocator(5))

ax[0,1].stackplot(x,np.array(counts_testset_partial_shared[::2])/(Neuron_num/100),np.array(avg_all_partial_shared_counts[::2]-counts_testset_partial_shared[::2])/(Neuron_num/100),colors=['#37A2FF','#A17DB4'], alpha=0.5)

ax[0,1].set_title('Partial-shared',fontsize=7,fontweight='bold')
ax[0,1].tick_params(labelsize=8)  
ax[0,1].xaxis.set_major_locator(ticker.MultipleLocator(5))

ax[1,0].stackplot(x,np.array(counts_testset_specific[::2])/(Neuron_num/100),np.array(avg_specific_counts[::2]-counts_testset_specific[::2])/(Neuron_num/100),colors=['#37A2FF','#A17DB4'], alpha=0.5)

ax[1,0].set_title('Specific',fontsize=7,fontweight='bold')
ax[1,0].tick_params(labelsize=8)  
ax[1,0].xaxis.set_major_locator(ticker.MultipleLocator(5))

ax[1,1].stackplot(x,np.array(counts_testset_non[::2])/(Neuron_num/100),np.array(avg_non_counts[::2]-counts_testset_non[::2])/(Neuron_num/100),colors=['#37A2FF','#A17DB4'], alpha=0.5)

ax[1,1].set_title('Non-activated',fontsize=7,fontweight='bold')
ax[1,1].tick_params(labelsize=8)  
ax[1,1].xaxis.set_major_locator(ticker.MultipleLocator(5))



fig.text(0.53, 0, 'Layers', ha='center',fontsize=8,fontweight='bold')
fig.text(0.03, 0.5, 'Percentage', va='center', rotation='vertical',fontsize=8,fontweight='bold')
fig.subplots_adjust(wspace=0.25,hspace=0.35) 

plt.show()

## across every token

In [ ]:
def get_out_all_bloomz(model, prompt, device): 

    model.eval()
    MLP_act = [f"transformer.h.{i}.mlp.gelu_impl" for i in range(30)]   
    with torch.no_grad():
        with TraceDict(model, MLP_act) as ret:
            output = model(prompt, output_hidden_states = True,output_attentions=True)
        MLP_act_value = [ret[act_value].output[0].detach().cpu().numpy()  for act_value in MLP_act]

    return MLP_act_value

In [ ]:
from numpy import *
import torch
import numpy as np


from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import matplotlib.pyplot as plt
from matplotlib.pyplot import MultipleLocator
import matplotlib.ticker as ticker



question_all,answer_all = load_data_XNLI()

test_counts = len(question_all)
langs_counts = len(question_all[0])


counts_each, counts_inter, counts_en_sub, counts_sub_en = [],[],[],[]    
mlp_act_all_sum = []
non_counts = []
for ind in range(test_counts):
    sents = question_all[ind]
    mlp_acts = []
    for sent in sents:
        encodings = tokenizer(sent, return_tensors='pt')
        input_ids = encodings['input_ids'].to('cuda')
        attention_mask = encodings['attention_mask'].to('cuda')
        mlp_act = get_out_all_bloomz(model,input_ids,model.device)
        mlp_act = np.array(mlp_act)
        mlp_act = (mlp_act>0).astype(int)

        mlp_act_sum = np.sum(mlp_act,axis=1)
        mlp_act_all_sum.append(mlp_act_sum)

    mlp_all = np.sum(mlp_act_all_sum,axis=0)  
    non,non_count = [],[]    
    for ly in range(LAYERS):
        mlp_non_inter = torch.nonzero(torch.tensor(mlp_all[ly]==0)).squeeze(1).detach().cpu().numpy()
        non.append(mlp_non_inter)  
        non_count.append(len(mlp_non_inter))
    non_counts.append(non_count)

non_counts = mean(non_counts, axis=0)   
print(np.array(non_counts).shape)


legend_font = {
    'style': 'normal',
    'size': 6.5, 
    'weight': "bold",  
}

legend_elements = [Line2D([0], [0], marker='s', color=colors[0], label='non-activated')]


fig, ax = plt.subplots(1,1,figsize=(2,2))

xLabel=[i+1 for i in range(LAYERS//2)]
x = [(i+1)*2 for i in range(LAYERS//2)]


ax.plot(x,np.array(non_counts[::2])/163.84,colors[0],marker='s',markersize='2.5') 
ax.tick_params(labelsize=8)    
ax.xaxis.set_major_locator(ticker.MultipleLocator(5))  


fig.text(0.53, -0.05, 'Layers', ha='center',fontsize=8,fontweight='bold')
fig.text(-0.1, 0.5, 'Percentage', va='center', rotation='vertical',fontsize=8,fontweight='bold')

fig.legend(handles=legend_elements, bbox_to_anchor=(0.2, -0.07), loc=2, ncol=5,prop=legend_font)

plt.show()

## over threshold

In [ ]:
from numpy import *
import torch
import numpy as np

from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import matplotlib.pyplot as plt
from matplotlib.pyplot import MultipleLocator
import matplotlib.ticker as ticker


question_all,answer_all = load_data_XNLI()

test_counts = len(question_all)
langs_counts = len(question_all[0])

thres_all_shared,thres_partial_shared,thres_non,thres_specific = [],[],[],[]
for thres in [0,0.1,0.2,0.3,0.4,0.5]:
    all_activated_counts,all_shared_counts,non_counts,specific_counts,all_partial_shared_counts = [],[],[],[] ,[]
    for ind in range(test_counts):
        sents = question_all[ind]
        mlp_acts = []
        for lang in range(len(sents)):
            question = sents[lang]
            encodings = tokenizer(question, return_tensors='pt')
            input_ids = encodings['input_ids'].to('cuda')

            if 'bloom' in model_name:
                mlp_act = act_bloom(input_ids)
            mlp_acts.append(mlp_act)

        mlp_all_act = []
        for i in range(langs_counts):
            mlp_act = (mlp_acts[i]>thres).astype(int)
            mlp_all_act.append(mlp_act)
        mlp_all = np.sum(mlp_all_act,axis=0)    

        all_activated,all_activated_count = [],[]
        for i in range(langs_counts):   
            activated,activated_count = [],[]
            for ly in range(LAYERS):
                act = torch.nonzero(torch.tensor(mlp_all_act[i][ly]==1)).squeeze(1).detach().cpu().numpy()
                activated.append(act) 
                activated_count.append(len(act))
            all_activated.append(activated)
            all_activated_count.append(activated_count)


        all_shared,all_shared_count = [],[]
        for ly in range(LAYERS):
            mlp_all_act_inter = torch.nonzero(torch.tensor(mlp_all[ly]==langs_counts)).squeeze(1).detach().cpu().numpy()
            all_shared.append(mlp_all_act_inter)
            all_shared_count.append(len(mlp_all_act_inter))

        non,non_count = [],[]
        mlp_all = np.sum(mlp_all_act,axis=0)
        for ly in range(LAYERS):
            mlp_all_act_inter = torch.nonzero(torch.tensor(mlp_all[ly]==0)).squeeze(1).detach().cpu().numpy()
            non.append(mlp_all_act_inter)  
            non_count.append(len(mlp_all_act_inter))


        specific,specific_count = [],[]
        for i in range(langs_counts):       
            indices_arr1 = np.where(mlp_all_act[i] == 1)
            indices_arr2 = np.where(mlp_all == 1)
            set_indices_arr1 = set(zip(indices_arr1[0], indices_arr1[1]))
            set_indices_arr2 = set(zip(indices_arr2[0], indices_arr2[1]))
            intersection = set_indices_arr1.intersection(set_indices_arr2)
            if len(intersection) > 0:
                rows_intersection, cols_intersection = zip(*intersection)
            else:
                rows_intersection, cols_intersection = [],[]
            row = [[] for _ in range(LAYERS)]
            for k in range(len(rows_intersection)):
                r,c = rows_intersection[k],cols_intersection[k]
                row[r].append(c)
            specific.append(row)


        all_specific,specific_count = [],[]
        for ly in range(LAYERS):
            temp = []
            for i in range(langs_counts):
                temp += specific[i][ly]
            all_specific.append(temp)
            specific_count.append(len(temp))

        partial_shared = []
        for lg in range(langs_counts):
            lg_some = []
            for ly in range(LAYERS):
                other = all_shared[ly].tolist() + specific[lg][ly]
                some = list((set(all_activated[lg][ly])-set(other)))
                lg_some.append(some)
            partial_shared.append(lg_some)

        all_partial_shared,all_partial_shared_count = [],[]
        for ly in range(LAYERS):
            temp = []
            for i in range(langs_counts):
                temp += partial_shared[i][ly]
            all_partial_shared.append(list(set(temp)))
            all_partial_shared_count.append(len(list(set(temp))))

        all_activated_counts.append(all_activated_count)
        all_shared_counts.append(all_shared_count)
        non_counts.append(non_count)
        specific_counts.append(specific_count)
        all_partial_shared_counts.append(all_partial_shared_count)



    all_activated_counts,all_shared_counts,non_counts,specific_counts,all_partial_shared_counts = mean(all_activated_counts, axis=0),mean(all_shared_counts, axis=0),mean(non_counts, axis=0),mean(specific_counts, axis=0),mean(all_partial_shared_counts, axis=0)

    thres_all_shared.append(all_shared_counts)
    thres_partial_shared.append(all_partial_shared_counts)
    thres_non.append(non_counts)
    thres_specific.append(specific_counts)
print(np.array(thres_all_shared).shape)


legend_font = {
'style': 'normal',
'size': 8,
'weight': "bold",  
}
thres = [0,0.1,0.2,0.3,0.4,0.5]

legend_elements = [
            Line2D([0], [0], marker='d', color=colors[1], label='all-shared'),
            Line2D([0], [0], marker='<', color=colors[2], label='partial-shared'),
            Line2D([0], [0], marker='o', color=colors[3], label='specific'),
            # Line2D([0], [0], marker='s', color=colors[0], label='non-activated'),
                  ]


fig, ax = plt.subplots(2,3,figsize=(5,4))

xLabel=[i+1 for i in range(LAYERS//2)]
x = [(i+1)*2 for i in range(LAYERS//2)]



ax[0,0].plot(x,np.array(thres_all_shared[0][::2])/163.84,colors[1],marker='d',markersize='2.5')
ax[0,0].plot(x,np.array(thres_partial_shared[0][::2])/163.84,colors[2],marker='<',markersize='2.5')
ax[0,0].plot(x,np.array(thres_specific[0][::2])/163.84,colors[3],marker='o',markersize='2.5')
# ax[0,0].plot(x,np.array(thres_non[0][::2])/163.84,colors[0],marker='s',markersize='2.5')
ax[0,0].set_title(f'Threshold 0',fontsize=7,fontweight='bold')
ax[0,0].tick_params(labelsize=8)  
ax[0,0].xaxis.set_major_locator(ticker.MultipleLocator(5))

ax[0,1].plot(x,np.array(thres_all_shared[1][::2])/163.84,colors[1],marker='d',markersize='2.5')
ax[0,1].plot(x,np.array(thres_partial_shared[1][::2])/163.84,colors[2],marker='<',markersize='2.5')
ax[0,1].plot(x,np.array(thres_specific[1][::2])/163.84,colors[3],marker='o',markersize='2.5')
# ax[0,1].plot(x,np.array(thres_non[1][::2])/163.84,colors[0],marker='s',markersize='2.5')
ax[0,1].set_title(f'Threshold 0.1',fontsize=7,fontweight='bold')
ax[0,1].tick_params(labelsize=8)  
ax[0,1].xaxis.set_major_locator(ticker.MultipleLocator(5))

ax[0,2].plot(x,np.array(thres_all_shared[2][::2])/163.84,colors[1],marker='d',markersize='2.5')
ax[0,2].plot(x,np.array(thres_partial_shared[2][::2])/163.84,colors[2],marker='<',markersize='2.5')
ax[0,2].plot(x,np.array(thres_specific[2][::2])/163.84,colors[3],marker='o',markersize='2.5')
# ax[0,2].plot(x,np.array(thres_non[2][::2])/163.84,colors[0],marker='s',markersize='2.5')
ax[0,2].set_title(f'Threshold 0.2',fontsize=7,fontweight='bold')
ax[0,2].tick_params(labelsize=8)  
ax[0,2].xaxis.set_major_locator(ticker.MultipleLocator(5))

ax[1,0].plot(x,np.array(thres_all_shared[3][::2])/163.84,colors[1],marker='d',markersize='2.5')
ax[1,0].plot(x,np.array(thres_partial_shared[3][::2])/163.84,colors[2],marker='<',markersize='2.5')
ax[1,0].plot(x,np.array(thres_specific[3][::2])/163.84,colors[3],marker='o',markersize='2.5')
# ax[1,0].plot(x,np.array(thres_non[3][::2])/163.84,colors[0],marker='s',markersize='2.5')
ax[1,0].set_title(f'Threshold 0.3',fontsize=7,fontweight='bold')
ax[1,0].tick_params(labelsize=8)  
ax[1,0].xaxis.set_major_locator(ticker.MultipleLocator(5))

ax[1,1].plot(x,np.array(thres_all_shared[4][::2])/163.84,colors[1],marker='d',markersize='2.5')
ax[1,1].plot(x,np.array(thres_partial_shared[4][::2])/163.84,colors[2],marker='<',markersize='2.5')
ax[1,1].plot(x,np.array(thres_specific[4][::2])/163.84,colors[3],marker='o',markersize='2.5')
# ax[1,1].plot(x,np.array(thres_non[4][::2])/163.84,colors[0],marker='s',markersize='2.5')
ax[1,1].set_title(f'Threshold 0.4',fontsize=7,fontweight='bold')
ax[1,1].tick_params(labelsize=8)  
ax[1,1].xaxis.set_major_locator(ticker.MultipleLocator(5))

ax[1,2].plot(x,np.array(thres_all_shared[5][::2])/163.84,colors[1],marker='d',markersize='2.5')
ax[1,2].plot(x,np.array(thres_partial_shared[5][::2])/163.84,colors[2],marker='<',markersize='2.5')
ax[1,2].plot(x,np.array(thres_specific[5][::2])/163.84,colors[3],marker='o',markersize='2.5')
# ax[1,2].plot(x,np.array(thres_non[5][::2])/163.84,colors[0],marker='s',markersize='2.5')
ax[1,2].set_title(f'Threshold 0.5',fontsize=7,fontweight='bold')
ax[1,2].tick_params(labelsize=8)  
ax[1,2].xaxis.set_major_locator(ticker.MultipleLocator(5))


fig.text(0.53, 0.03, 'Layers', ha='center',fontsize=8,fontweight='bold')
fig.text(0.03, 0.5, 'Percentage', va='center', rotation='vertical',fontsize=8,fontweight='bold')
fig.subplots_adjust(wspace=0.25,hspace=0.35) 

fig.legend(handles=legend_elements, bbox_to_anchor=(0.15, -0.05), loc=3, ncol=3,prop=legend_font)

plt.show()